In [16]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
import sasc.viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### S02 comparisons

In [29]:
# gemv_flatmaps_pilot = joblib.load(join(
#     RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
# gemv_flatmaps_pilot5 = joblib.load(join(
#     RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
# gemv_flatmaps_dict = gemv_flatmaps_pilot | gemv_flatmaps_pilot5

gemv_flatmaps_dict = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps_all", 'UTS03', 'default', 'resps_avg_dict_pilot3.pkl'))

In [30]:
gemv_flatmaps[('age', np.int64(109))].shape

(95556,)

In [31]:
gemv_flatmaps_dict.keys()

normalize_flatmaps = True
if normalize_flatmaps:
    for k, v in gemv_flatmaps_dict.items():
        flatmap_unnormalized = gemv_flatmaps_dict[k]
        gemv_flatmaps_dict[k] = (
            flatmap_unnormalized - flatmap_unnormalized.mean()) / flatmap_unnormalized.std()

In [32]:
subject = 'S03'
suffix_setting = '_fedorenko'
# subject = 'S03'
# rois_dict = joblib.load(join(regions_idxs_dir, f'rois_{subject}.jbl'))
# rois = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/', 'communication_rois_UTS02.jbl'))
rois = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/',
                   f'communication_rois_v2_UT{subject}.jbl'))
rois_dict_raw = {i: rois[i] for i in range(len(rois))}
# custom merge contralateral regions
if subject == 'S02':
    raw_idxs = [
        [0, 7],
        [3, 4],
        [1, 5],
        [2, 6],
    ]
elif subject == 'S03':
    raw_idxs = [
        [0, 7],
        [3, 4],
        [2, 5],
        [1, 6],
    ]
rois_dict = {
    i: np.vstack([rois_dict_raw[j] for j in idxs]).sum(axis=0)
    for i, idxs in enumerate(raw_idxs)
}
if suffix_setting == '_fedorenko':
    rois_fedorenko = joblib.load(join(
        FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/', 'lang_localizer_UTS03.jbl'))
    rois_dict = {
        i: rois_fedorenko[i] for i in range(len(rois_fedorenko))
    }

In [33]:
gemv_flatmaps_dict.keys()
avg_defaultdict = defaultdict(list)
for roi_idx, roi in rois_dict.items():
    for explanation in gemv_flatmaps_dict.keys():
        avg_defaultdict[roi_idx].append(
            np.mean(gemv_flatmaps_dict[explanation][roi > 0]))
        # corrs_defaultdict[roi_idx].append(
        # np.corrcoef(roi, gemv_flatmaps_dict[explanation])[0, 1])

In [34]:
df = pd.DataFrame(avg_defaultdict, index=gemv_flatmaps_dict.keys())
df.columns = [f'ROI_{i}' for i in rois_dict.keys()]
df
# add mean col
df['AVG'] = df.mean(axis=1)
df = df.round(3).sort_values('AVG', ascending=False)
# display pandas formatting with redblue colormap centered at 0 rounded to 3 decimal places
vabs = max(abs(df.values.min()), abs(df.values.max()))
df.style.background_gradient(cmap='coolwarm', axis=None, vmax=vabs, vmin=-vabs).format("{:.3f}").set_caption(
    'Average GEM-V driving response averaged over ROI'
)

,,ROI_0,ROI_1,ROI_2,ROI_3,ROI_4,AVG
love and joy,337,1.050,1.801,1.053,0.987,1.499,1.278
agreement and questioning,458,0.698,1.059,1.027,0.919,1.072,0.955
conflict resolution,173,0.895,1.175,0.973,0.824,0.858,0.945
food and drinks,466,0.421,0.700,1.215,1.257,0.270,0.773
numbers,9,0.383,0.913,0.878,0.645,0.904,0.745
age,160,0.519,0.696,0.362,0.700,0.976,0.651
"vomiting, sickness",395,0.425,0.655,0.701,0.813,0.317,0.582
body language,99,0.009,0.452,0.574,0.669,1.029,0.547
locations,342,0.362,0.046,1.086,0.799,0.406,0.540
age,109,0.840,0.566,0.325,0.141,0.496,0.474


In [ ]:
sasc.viz._save_flatmap(
    gemv_flatmaps_dict[('relationships between people', 'qa')], subject, fname_save=f'gemv_flatmaps/relationships_{subject}.png')

In [36]:
from sasc import config

In [ ]:
stories_data_dict = joblib.load(
    join(config.RESULTS_DIR, 'processed', 'pilot3_story_data.pkl'))

In [48]:
pd.DataFrame(stories_data_dict).iloc[0]['prompts']

['Write the beginning paragraph of a long, coherent story. The story should be about "love and joy". Make sure it contains several words related to "love and joy", such as "love love me", "joyous joyous", "was joyous joyous".',
 'Write the next paragraph of the story, staying consistent with the story so far, but now make it about "age". Make sure it contains several words related to "age", such as "i was seventeen", "i was eighteen", "me was nineteen".',
 'Write the next paragraph of the story, staying consistent with the story so far, but now make it about "movement or action". Make sure it contains several words related to "movement or action", such as "neighbor came over", "seen me walking", "husband just walked".',
 'Write the next paragraph of the story, staying consistent with the story so far, but now make it about "numbers or measurements". Make sure it contains several words related to "numbers or measurements", such as "four hundred and", "five and nineteen", "six hundred an